In [18]:

import csv
import random
from faker import Faker

fake = Faker()

def generate_synthetic_employees(num_employees):
    employees = []
    for i in range(num_employees):
        vorname = fake.first_name()  # Generieren eines zufälligen Vornamens
        nachname = fake.last_name()  # Generieren eines zufälligen Nachnamens
        name = f"{vorname} {nachname}".upper()  # Zufälliger Name in Großbuchstaben
        alter = random.randint(20, 60)

        # Berechnung der Betriebszugehörigkeit
        max_betriebszugehoerigkeit = alter - 16
        if max_betriebszugehoerigkeit < 0:
            max_betriebszugehoerigkeit = 0
        betriebszugehoerigkeit = random.randint(1, max_betriebszugehoerigkeit)
        
        ehepartner_unterhalt = random.choice([0, 1])  # Zufällige Auswahl von 0 (keine Unterhaltspflicht) oder 1 (Unterhaltspflicht)
        kinder_unterhalt = random.randint(0, 3)  # Zufällige Anzahl von unterhaltsberechtigten Kindern zwischen 0 und 3
        schwerbehindert = random.choice([0, 1])  # Zufällige Auswahl von 0 (nicht schwerbehindert) oder 1 (schwerbehindert)
        arbeitsvertrag = random.choice([80, 130, 145, 160, 167])
        employee_data = [vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert, arbeitsvertrag]
        employees.append(employee_data)
    return employees

# Generiere synthetische Daten für 40 fiktive Mitarbeiter
num_employees = 40
synthetic_employees = generate_synthetic_employees(num_employees)
filename = 'new_names_synthetic_employees.csv'

# Schreibe die synthetischen Mitarbeiterdaten in eine CSV-Datei mit 7 Spalten
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Schreibe die Kopfzeile
    writer.writerow(["Vorname", "Nachname","Alter", "Betriebszugehoerigkeit", "EhepartnerUnterhalt", "KinderUnterhalt", "Schwerbehindert", "Arbeitsvertrag"])
    # Schreibe die Daten
    writer.writerows(synthetic_employees)

print(f"Die Datei {filename} wurde erfolgreich erstellt.")


Die Datei new_names_synthetic_employees.csv wurde erfolgreich erstellt.


In [19]:
import csv
import pandas as pd

class Mitarbeiter:
    def __init__(self, vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert):
        self.vorname = vorname
        self.nachname = nachname
        self.alter = alter
        self.betriebszugehoerigkeit = betriebszugehoerigkeit
        self.ehepartner_unterhalt = ehepartner_unterhalt
        self.kinder_unterhalt = kinder_unterhalt
        self.schwerbehindert = schwerbehindert
        self.punkte = alter + 1.5 * betriebszugehoerigkeit + 5 * ehepartner_unterhalt + 5 * kinder_unterhalt + 8 * schwerbehindert
        self.dismiss = False
        self.gruppe = None

def read_data(filename):
    mitarbeiter_list = []
    with open(filename, 'r', newline='') as file:
        reader = csv.DictReader(file, delimiter='\t')
        
        # Bereinigen der Spaltennamen
        reader.fieldnames = [field.strip() for field in reader.fieldnames]
        
        for row in reader:
            vorname = row['Vorname'].strip()
            nachname = row['Nachname'].strip()
            alter = int(row['Alter'].strip())
            betriebszugehoerigkeit = int(row['Betriebszugehoerigkeit'].strip())
            ehepartner_unterhalt = int(row['EhepartnerUnterhalt'].strip())
            kinder_unterhalt = int(row['KinderUnterhalt'].strip())
            schwerbehindert = int(row['Schwerbehindert'].strip())
            mitarbeiter = Mitarbeiter(vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert)
            mitarbeiter_list.append(mitarbeiter)
           
    return mitarbeiter_list

def assign_groups(mitarbeiter_list):
    groups = {1: [], 2: [], 3: [], 4: []}
    for mitarbeiter in mitarbeiter_list:
        if mitarbeiter.alter <= 30:
            group = 1
        elif 31 <= mitarbeiter.alter <= 40:
            group = 2
        elif 41 <= mitarbeiter.alter <= 50:
            group = 3
        else:
            group = 4
        mitarbeiter.gruppe = group
        groups[group].append(mitarbeiter)
        
    return groups

def mark_for_dismissal(groups, total_dismissals):
    dismissals_per_group = total_dismissals // 4
    remaining_dismissals = total_dismissals - dismissals_per_group * 4

    for group, mitarbeiter_list in groups.items():
        sorted_mitarbeiter = sorted(mitarbeiter_list, key=lambda x: x.punkte)
        actual_dismissals = min(dismissals_per_group, len(sorted_mitarbeiter))
        for i in range(actual_dismissals):
            sorted_mitarbeiter[i].dismiss = True

    remaining_mitarbeiter = [mitarbeiter for sublist in groups.values() for mitarbeiter in sublist if not mitarbeiter.dismiss]
    sorted_remaining_mitarbeiter = sorted(remaining_mitarbeiter, key=lambda x: x.punkte)
    for i in range(min(remaining_dismissals, len(sorted_remaining_mitarbeiter))):
        sorted_remaining_mitarbeiter[i].dismiss = True

def create_results_dataframe(groups):
    data = []
    for group, mitarbeiter_list in groups.items():
        # Sortieren der Mitarbeiter nach Punkten in absteigender Reihenfolge
        sorted_mitarbeiter_list = sorted(mitarbeiter_list, key=lambda x: x.punkte, reverse=True)
        for mitarbeiter in sorted_mitarbeiter_list:
            data.append([
                mitarbeiter.vorname,
                mitarbeiter.nachname,
                mitarbeiter.alter,
                mitarbeiter.betriebszugehoerigkeit,
                mitarbeiter.ehepartner_unterhalt,
                mitarbeiter.kinder_unterhalt,
                mitarbeiter.schwerbehindert,
                mitarbeiter.punkte,
                "Ja" if mitarbeiter.dismiss else "Nein"
            ])
    
    df = pd.DataFrame(data, columns=[
        "Vorname", "Nachname", "Alter", "Betriebszugehörigkeit", 
        "EhepartnerUnterhalt", "KinderUnterhalt", "Schwerbehindert", 
        "Punkte", "Entlassungsempfehlung"
    ])
    return df

def main():
    filename = 'names_synthetic_employees.csv'
    total_dismissals = 10
    mitarbeiter_list = read_data(filename)
    if mitarbeiter_list:
        groups = assign_groups(mitarbeiter_list)
        mark_for_dismissal(groups, total_dismissals)
        results_df = create_results_dataframe(groups)
        print(results_df)
        
        # Speichern der Ergebnisse in eine neue CSV-Datei
        results_df.to_csv('mitarbeiter_ergebnisse.csv', index=False)

if __name__ == "__main__":
    main()


KeyError: 'Vorname'

# Code mit Arbeitszeitberücksichtigung

In [7]:
import csv
import pandas as pd

class Mitarbeiter:
    def __init__(self, vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert, arbeitsvertrag):
        self.vorname = vorname
        self.nachname = nachname
        self.alter = alter
        self.betriebszugehoerigkeit = betriebszugehoerigkeit
        self.ehepartner_unterhalt = ehepartner_unterhalt
        self.kinder_unterhalt = kinder_unterhalt
        self.schwerbehindert = schwerbehindert
        self.arbeitsvertrag = arbeitsvertrag
        self.punkte = alter + 1.5 * betriebszugehoerigkeit + 5 * ehepartner_unterhalt + 5 * kinder_unterhalt + 8 * schwerbehindert
        self.dismiss = False
        self.gruppe = None

def read_data(filename):
    mitarbeiter_list = []
    with open(filename, 'r', newline='') as file:
        reader = csv.DictReader(file, delimiter='\t')
        
        # Bereinigen der Spaltennamen
        reader.fieldnames = [field.strip() for field in reader.fieldnames]
        
        for row in reader:
            vorname = row['Vorname'].strip()
            nachname = row['Nachname'].strip()
            alter = int(row['Alter'].strip())
            betriebszugehoerigkeit = int(row['Betriebszugehoerigkeit'].strip())
            ehepartner_unterhalt = int(row['EhepartnerUnterhalt'].strip())
            kinder_unterhalt = int(row['KinderUnterhalt'].strip())
            schwerbehindert = int(row['Schwerbehindert'].strip())
            arbeitsvertrag = int(row['Arbeitsvertrag'].strip())
            mitarbeiter = Mitarbeiter(vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert, arbeitsvertrag)
            mitarbeiter_list.append(mitarbeiter)
           
    return mitarbeiter_list

def assign_groups(mitarbeiter_list):
    groups = {1: [], 2: [], 3: [], 4: []}
    for mitarbeiter in mitarbeiter_list:
        if mitarbeiter.alter <= 30:
            group = 1
        elif 31 <= mitarbeiter.alter <= 40:
            group = 2
        elif 41 <= mitarbeiter.alter <= 50:
            group = 3
        else:
            group = 4
        mitarbeiter.gruppe = group
        groups[group].append(mitarbeiter)
        
    return groups

def mark_for_dismissal(groups, target_hours):
    total_hours_to_dismiss = target_hours
    dismissals_per_group = total_hours_to_dismiss / 4

    remaining_mitarbeiter = [mitarbeiter for sublist in groups.values() for mitarbeiter in sublist]
    sorted_remaining_mitarbeiter = sorted(remaining_mitarbeiter, key=lambda x: x.punkte)

    dismissed_hours = 0
    for mitarbeiter in sorted_remaining_mitarbeiter:
        if dismissed_hours >= total_hours_to_dismiss:
            break
        mitarbeiter.dismiss = True
        dismissed_hours += mitarbeiter.arbeitsvertrag

def create_results_dataframe(groups):
    data = []
    for group, mitarbeiter_list in groups.items():
        # Sortieren der Mitarbeiter nach Punkten in absteigender Reihenfolge
        sorted_mitarbeiter_list = sorted(mitarbeiter_list, key=lambda x: x.punkte, reverse=True)
        for mitarbeiter in sorted_mitarbeiter_list:
            data.append([
                mitarbeiter.vorname,
                mitarbeiter.nachname,
                mitarbeiter.alter,
                mitarbeiter.betriebszugehoerigkeit,
                mitarbeiter.ehepartner_unterhalt,
                mitarbeiter.kinder_unterhalt,
                mitarbeiter.schwerbehindert,
                mitarbeiter.arbeitsvertrag,
                mitarbeiter.punkte,
                "Ja" if mitarbeiter.dismiss else "Nein"
            ])
    
    df = pd.DataFrame(data, columns=[
        "Vorname", "Nachname", "Alter", "Betriebszugehörigkeit", 
        "EhepartnerUnterhalt", "KinderUnterhalt", "Schwerbehindert", 
        "Arbeitsvertrag", "Punkte", "Entlassungsempfehlung"
    ])
    return df

def main():
    filename = 'names_synthetic_employees.csv'
    target_hours = 1670
    mitarbeiter_list = read_data(filename)
    if mitarbeiter_list:
        groups = assign_groups(mitarbeiter_list)
        mark_for_dismissal(groups, target_hours)
        results_df = create_results_dataframe(groups)
        print(results_df)
        
        # Speichern der Ergebnisse in eine neue CSV-Datei
        results_df.to_csv('mitarbeiter_ergebnisse.csv', index=False)

if __name__ == "__main__":
    main()


       Vorname    Nachname  Alter  Betriebszugehörigkeit  EhepartnerUnterhalt  \
0        EMILY       KLEIN     23                     25                    1   
1    CHRISTINA        HILL     25                     22                    1   
2     VICTORIA       BURNS     27                     16                    0   
3      GREGORY         LEE     24                     16                    0   
4         JOHN       SMITH     23                     18                    1   
5          AMY        COBB     25                     17                    1   
6       ELAINE        CHEN     25                      5                    1   
7     DANIELLE       DOYLE     27                      8                    1   
8      CAITLIN      MARTIN     23                      1                    1   
9        JASON    GRIFFITH     32                     13                    1   
10       DEBRA      POWELL     36                     18                    0   
11        JUDY       LLOYD  

# Angepasstes Alter

In [11]:
import csv
import pandas as pd

class Mitarbeiter:
    def __init__(self, vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert):
        self.vorname = vorname
        self.nachname = nachname
        self.alter = alter
        self.betriebszugehoerigkeit = self.adjust_seniority(alter, betriebszugehoerigkeit)
        self.ehepartner_unterhalt = ehepartner_unterhalt
        self.kinder_unterhalt = kinder_unterhalt
        self.schwerbehindert = schwerbehindert
        self.punkte = alter + 1.5 * self.betriebszugehoerigkeit + 5 * ehepartner_unterhalt + 5 * kinder_unterhalt + 8 * schwerbehindert
        self.dismiss = False
        self.gruppe = None

    def adjust_seniority(self, alter, betriebszugehoerigkeit):
        eintrittsalter = alter - betriebszugehoerigkeit
        if eintrittsalter < 16:
            betriebszugehoerigkeit = alter - 16
        return max(betriebszugehoerigkeit, 0)  # Ensure seniority is non-negative

def read_data(filename):
    mitarbeiter_list = []
    with open(filename, 'r', newline='') as file:
        reader = csv.DictReader(file, delimiter='\t')
        
        # Bereinigen der Spaltennamen
        reader.fieldnames = [field.strip() for field in reader.fieldnames]
        
        for row in reader:
            vorname = row['Vorname'].strip()
            nachname = row['Nachname'].strip()
            alter = int(row['Alter'].strip())
            betriebszugehoerigkeit = int(row['Betriebszugehoerigkeit'].strip())
            ehepartner_unterhalt = int(row['EhepartnerUnterhalt'].strip())
            kinder_unterhalt = int(row['KinderUnterhalt'].strip())
            schwerbehindert = int(row['Schwerbehindert'].strip())
            mitarbeiter = Mitarbeiter(vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert)
            mitarbeiter_list.append(mitarbeiter)
                
    return mitarbeiter_list

def assign_groups(mitarbeiter_list):
    groups = {1: [], 2: [], 3: [], 4: []}
    for mitarbeiter in mitarbeiter_list:
        if mitarbeiter.alter <= 30:
            group = 1
        elif 31 <= mitarbeiter.alter <= 40:
            group = 2
        elif 41 <= mitarbeiter.alter <= 50:
            group = 3
        else:
            group = 4
        mitarbeiter.gruppe = group
        groups[group].append(mitarbeiter)
        
    return groups

def mark_for_dismissal(groups, total_dismissals):
    dismissals_per_group = total_dismissals // 4
    remaining_dismissals = total_dismissals - dismissals_per_group * 4

    for group, mitarbeiter_list in groups.items():
        sorted_mitarbeiter = sorted(mitarbeiter_list, key=lambda x: x.punkte)
        actual_dismissals = min(dismissals_per_group, len(sorted_mitarbeiter))
        for i in range(actual_dismissals):
            sorted_mitarbeiter[i].dismiss = True

    remaining_mitarbeiter = [mitarbeiter for sublist in groups.values() for mitarbeiter in sublist if not mitarbeiter.dismiss]
    sorted_remaining_mitarbeiter = sorted(remaining_mitarbeiter, key=lambda x: x.punkte)
    for i in range(min(remaining_dismissals, len(sorted_remaining_mitarbeiter))):
        sorted_remaining_mitarbeiter[i].dismiss = True

def create_results_dataframe(groups):
    data = []
    for group, mitarbeiter_list in groups.items():
        for mitarbeiter in mitarbeiter_list:
            data.append([
                mitarbeiter.vorname,
                mitarbeiter.nachname,
                mitarbeiter.alter,
                mitarbeiter.betriebszugehoerigkeit,
                mitarbeiter.ehepartner_unterhalt,
                mitarbeiter.kinder_unterhalt,
                mitarbeiter.schwerbehindert,
                mitarbeiter.punkte,
                "Ja" if mitarbeiter.dismiss else "Nein"
            ])
    
    df = pd.DataFrame(data, columns=[
        "Vorname", "Nachname", "Alter", "Betriebszugehörigkeit", 
        "EhepartnerUnterhalt", "KinderUnterhalt", "Schwerbehindert", 
        "Punkte", "Entlassungsempfehlung"
    ])
    return df

def main():
    filename = 'new_names_synthetic_employees.csv'
    total_dismissals = 10
    mitarbeiter_list = read_data(filename)
    if mitarbeiter_list:
        groups = assign_groups(mitarbeiter_list)
        mark_for_dismissal(groups, total_dismissals)
        results_df = create_results_dataframe(groups)
        print(results_df)

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'new_names_synthetic_employees.csv'